In [0]:
%restart_python

In [0]:
silver_df = spark.table("workspace.default.silver_eco2mix_regional_tr_simple")

In [0]:
silver_df.display()

Overall load evolution.


In [0]:
from pyspark.sql import functions as F

daily_conso = (
    silver_df
    .groupBy("date")
    .agg(F.sum("consommation").alias("consommation_mwh"))
    .orderBy("date")
)

display(daily_conso)


Databricks visualization. Run in Databricks to view.

How each filière contributes per day.

In [0]:
daily_mix = (
    silver_df
    .groupBy("date")
    .agg(
        F.sum("thermique").alias("thermique_mwh"),
        F.sum("nucleaire").alias("nucleaire_mwh"),
        F.sum("eolien").alias("eolien_mwh"),
        F.sum("solaire").alias("solaire_mwh"),
        F.sum("hydraulique").alias("hydraulique_mwh"),
        F.sum("bioenergies").alias("bioenergies_mwh")
    )
    .orderBy("date")
)

display(daily_mix)


Databricks visualization. Run in Databricks to view.

Typical intraday profile (average quarter-hour)


In [0]:
intraday_profile = (
    silver_df
    .groupBy("heure")
    .agg(
        F.avg("consommation").alias("consommation_moy_mw"),
        F.avg("eolien").alias("eolien_moy_mw"),
        F.avg("solaire").alias("solaire_moy_mw")
    )
    .orderBy("heure")
)

display(intraday_profile)

Databricks visualization. Run in Databricks to view.

Impact des heures pleines / heures creuses

In [0]:
hp_hc_stats = (
    silver_df
    .groupBy("is_heures_pleines", "periode_journee")
    .agg(
        F.avg("consommation").alias("consommation_moy_mw"),
        F.max("consommation").alias("consommation_max_mw"),
        F.avg("thermique").alias("thermique_moy_mw"),
        F.avg("eolien").alias("eolien_moy_mw"),
        F.avg("solaire").alias("solaire_moy_mw")
    )
    .orderBy("is_heures_pleines", "periode_journee")
)

display(hp_hc_stats)

Databricks visualization. Run in Databricks to view.

Kepping the timeseries data for ML study

In [0]:
cols_to_keep = [
    "date",
    "heure",
    "consommation",
    "year",
    "month",
    "day_of_month",
    "hour_of_day",
    "minute_of_hour",
]

df_gold_timeseries = silver_df.select(*cols_to_keep)

df_gold_timeseries.printSchema()


In [0]:
(
    df_gold_timeseries
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable("workspace.default.gold_eco2mix_regional_tr_simple")
)


In [0]:
display(spark.read.table("workspace.default.gold_eco2mix_regional_tr_simple").toPandas().set_index("date"))